영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

model = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate


examples = [
    {
        "movie": "Avatar",
        "answer": """Director: James Cameron

Main Cast: Sam Worthington, Zoe Saldana, Stephen Lang, Michelle Rodriguez, Sigourney Weaver, Giovanni Ribisi, Joel David Moore, CCH Pounder, Wes Studi, Laz Alonso

Budget: $237 million

Box Office Revenue: $2.923 billion

Genre: Epic Science Fiction

Synopsis: Set in the mid-22nd century, Avatar tells the story of humanity's colonization of Pandora, a lush moon in the Alpha Centauri star system, for mining the valuable mineral unobtanium. The expansion threatens the existence of the Na'vi, a local tribe of humanoid species. The film follows Jake Sully, a paraplegic Marine who becomes part of the Avatar Program, where he controls a genetically engineered Na'vi body to interact with the natives. As Jake becomes deeply integrated into Na'vi culture, he finds himself in a conflict between following orders and protecting his new home
             """,
    },
    {
        "movie": "Transformers",
        "answer": """Director: Michael Bay

Main Cast: Shia LaBeouf, Tyrese Gibson, Josh Duhamel, Megan Fox, Rachael Taylor, John Turturro, Jon Voight

Budget: $150–200 million

Box Office Revenue: $709.7 million

Genre: Science Fiction, Action

Synopsis: A teenager, Sam Witwicky, becomes involved in a war between the Autobots and Decepticons, two factions of alien robots that can transform into everyday machinery. The Autobots seek the AllSpark, a powerful artifact, to rebuild their home planet Cybertron and end the war, while the Decepticons aim to use it to conquer the universe
        """,
    },
    {
        "movie": "Iron man",
        "answer": """Director: Jon Favreau

Main Cast: Robert Downey Jr., Terrence Howard, Jeff Bridges, Gwyneth Paltrow, Leslie Bibb, Shaun Toub

Budget: $130–140 million

Box Office Revenue: $585.8 million

Genre: Superhero

Synopsis: Tony Stark, a wealthy industrialist and master engineer, is taken captive by terrorists in Afghanistan. Utilizing his genius, Stark builds a mechanized suit of armor to escape, becoming the superhero Iron Man. The film explores Stark's transformation and the challenges he faces, including dealing with the consequences of his company's weapons manufacturing and confronting a betrayal within his own ranks
     """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [("human", "Please explain about the {movie}"), ("ai", "{answer}")]
)

few_shot_example = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a movie expert. When someone gives you a movie title, you respond with information about the movie, including the director, main cast, budget, box office revenue, genre, and a synopsis.",
        ),
        few_shot_example,
        ("human", "Please explain about the {movie}."),
    ]
)

chain = final_prompt | model

chain.invoke({"movie": "Titanic"})

Director: James Cameron

Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Frances Fisher, Bernard Hill, Jonathan Hyde, Danny Nucci, David Warner, Bill Paxton

Budget: $200 million

Box Office Revenue: $2.195 billion

Genre: Romance, Drama

Synopsis: Titanic is a fictionalized account of the sinking of the RMS Titanic, a luxurious ocean liner that struck an iceberg and sank on its maiden voyage in 1912. The film follows the love story between Jack Dawson, a penniless artist, and Rose DeWitt Bukater, a young socialite, who meet and fall in love aboard the ship. As the Titanic faces disaster, Jack and Rose must fight for survival amidst the chaos and tragedy unfolding around them. The film alternates between the present day, where an elderly Rose recounts her experience, and the events leading up to the ship's demise.

AIMessageChunk(content="Director: James Cameron\n\nMain Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Frances Fisher, Bernard Hill, Jonathan Hyde, Danny Nucci, David Warner, Bill Paxton\n\nBudget: $200 million\n\nBox Office Revenue: $2.195 billion\n\nGenre: Romance, Drama\n\nSynopsis: Titanic is a fictionalized account of the sinking of the RMS Titanic, a luxurious ocean liner that struck an iceberg and sank on its maiden voyage in 1912. The film follows the love story between Jack Dawson, a penniless artist, and Rose DeWitt Bukater, a young socialite, who meet and fall in love aboard the ship. As the Titanic faces disaster, Jack and Rose must fight for survival amidst the chaos and tragedy unfolding around them. The film alternates between the present day, where an elderly Rose recounts her experience, and the events leading up to the ship's demise.")